In [1]:
import requests

# This gives information for each station that remains stable over time
url_stations = 'https://gbfs.citibikenyc.com/gbfs/en/station_information.json'
# This gives the live status of all the stations (e.g., bikes available etc)
url_status = 'https://gbfs.citibikenyc.com/gbfs/en/station_status.json'



In [2]:
# This code creates a connection to the database
import MySQLdb as mdb
import sys

con = mdb.connect(host = 'localhost', 
                  user = 'root', 
                  passwd = 'dwdstudent2015', 
                  charset='utf8', use_unicode=True);

In [3]:
# Run a query to create a database that will hold the data
db_name = 'citibike_new'
create_db_query = "CREATE DATABASE IF NOT EXISTS {db} DEFAULT CHARACTER SET 'utf8'".format(db=db_name)

# Create a database
cursor = con.cursor()
cursor.execute(create_db_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:7: Warning: (1007, "Can't create database 'citibike_new'; database exists")
  import sys


In [4]:
# Create the two tables. One for storing the time-invariant station data
# and another table to store the time-varying station status data
cursor = con.cursor()
table_name = 'Stations'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (station_id int, 
                                name varchar(250), 
                                capacity int,
                                lat float,
                                lon float,
                                region_id int,
                                short_name varchar(250),
                                rental_url varchar(250),
                                eightd_has_key_dispenser bool,
                                PRIMARY KEY(station_id)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1050, "Table 'Stations' already exists")


In [5]:
# Create the time-varying table
cursor = con.cursor()
table_name = 'Status'
create_table_query = '''CREATE TABLE IF NOT EXISTS {db}.{table} 
                                (station_id int, 
                                last_reported datetime,
                                num_bikes_available int,
                                num_bikes_disabled int,
                                num_docks_available int,
                                num_docks_disabled int,
                                is_installed bool,
                                is_renting bool,
                                is_returning bool,
                                eightd_has_available_keys bool,
                                PRIMARY KEY(station_id, last_reported)
                                )'''.format(db=db_name, table=table_name)
cursor.execute(create_table_query)
cursor.close()

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: Warning: (1050, "Table 'Status' already exists")


In [6]:
# We fetch for now just the time-invariant data
# Notice that we have the INSERT IGNORE so that even when we add the same entry
# again, we do not get an error that the line exists. We do get warnings
# but this is expected
results = requests.get(url_stations).json() 
data = results["data"]["stations"]

table_name = 'Stations'
query_template = '''INSERT IGNORE INTO {db}.{table}(station_id, name,
                                        capacity, lat, lon,
                                        region_id,
                                        short_name,
                                        rental_url,
                                        eightd_has_key_dispenser) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

for entry in data:
    station_id = int(entry['station_id'])
    name = entry['name']
    capacity = entry['capacity']
    lat = entry['lat']
    lon = entry['lon']
    region_id = entry.get('region_id')
    short_name = entry['short_name']
    rental_url = entry['rental_url']
    eightd_has_key_dispenser = entry['eightd_has_key_dispenser']
                       
    print("Inserting station", station_id, "at", name)
    query_parameters = (station_id, name, capacity, lat, lon,
                        region_id, short_name, rental_url, eightd_has_key_dispenser)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

Inserting station 72 at W 52 St & 11 Ave
Inserting station 79 at Franklin St & W Broadway
Inserting station 82 at St James Pl & Pearl St
Inserting station 83 at Atlantic Ave & Fort Greene Pl
Inserting station 116 at W 17 St & 8 Ave
Inserting station 119 at Park Ave & St Edwards St
Inserting station 120 at Lexington Ave & Classon Ave
Inserting station 127 at Barrow St & Hudson St
Inserting station 128 at MacDougal St & Prince St
Inserting station 143 at Clinton St & Joralemon St
Inserting station 144 at Nassau St & Navy St
Inserting station 146 at Hudson St & Reade St
Inserting station 150 at E 2 St & Avenue C
Inserting station 151 at Cleveland Pl & Spring St
Inserting station 152 at Warren St & Church St
Inserting station 153 at E 40 St & 5 Ave
Inserting station 157 at Henry St & Atlantic Ave
Inserting station 161 at LaGuardia Pl & W 3 St
Inserting station 164 at E 47 St & 2 Ave
Inserting station 167 at E 39 St & 3 Ave
Inserting station 168 at W 18 St & 6 Ave
Inserting station 173 at B

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '72' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '79' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '82' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '83' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '116' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '119' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '120' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '127' for key 'PRIMARY'")
/usr/local/l

Inserting station 523 at W 38 St & 8 Ave
Inserting station 524 at W 43 St & 6 Ave
Inserting station 525 at W 34 St & 11 Ave
Inserting station 526 at E 33 St & 5 Ave
Inserting station 527 at E 33 St & 2 Ave
Inserting station 528 at 2 Ave & E 31 St
Inserting station 529 at W 42 St & 8 Ave
Inserting station 530 at 11 Ave & W 59 St
Inserting station 531 at Forsyth St & Broome St
Inserting station 532 at S 5 Pl & S 4 St
Inserting station 533 at Broadway & W 39 St
Inserting station 534 at Water - Whitehall Plaza
Inserting station 536 at 1 Ave & E 30 St
Inserting station 537 at Lexington Ave & E 24 St
Inserting station 539 at Metropolitan Ave & Bedford Ave
Inserting station 540 at Lexington Ave & E 29 St
Inserting station 545 at E 23 St & 1 Ave
Inserting station 546 at E 30 St & Park Ave S
Inserting station 2000 at Front St & Washington St
Inserting station 2001 at Sands St & Navy St
Inserting station 2002 at Wythe Ave & Metropolitan Ave
Inserting station 2003 at 1 Ave & E 18 St
Inserting sta

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '523' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '524' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '525' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '526' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '527' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '528' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '529' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '530' for key 'PRIMARY'")
/usr/loc

Inserting station 3327 at 3 Ave & E 100 St
Inserting station 3328 at W 100 St & Manhattan Ave
Inserting station 3329 at Degraw St & Smith St
Inserting station 3330 at Henry St & Bay St
Inserting station 3331 at Riverside Dr & W 104 St
Inserting station 3332 at Degraw St & Hoyt St
Inserting station 3333 at Columbia St & Lorraine St
Inserting station 3335 at Union St & 4 Ave
Inserting station 3336 at E 97 St & Madison Ave
Inserting station 3337 at Dwight St & Van Dyke St
Inserting station 3338 at 2 Ave & E 99 St
Inserting station 3339 at Berkeley Pl & 6 Ave
Inserting station 3340 at Wolcott St & Dwight St
Inserting station 3341 at Central Park West & W 102 St
Inserting station 3342 at Pioneer St & Richards St
Inserting station 3343 at W 107 St & Columbus Ave
Inserting station 3344 at Pioneer St & Van Brunt St
Inserting station 3345 at Madison Ave & E 99 St
Inserting station 3346 at Berkeley Pl & 7 Ave
Inserting station 3347 at Van Brunt St & Wolcott St
Inserting station 3348 at Coffey St

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3337' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3338' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3339' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3340' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3341' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3342' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3343' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3344' for key 'PRIMARY'")


Inserting station 3623 at Coming Soon: W 120 St & Claremont Ave
Inserting station 3624 at Coming Soon: 31 St & 39 Ave
Inserting station 3625 at Coming Soon: 28 St & 40 Ave
Inserting station 3626 at Coming Soon: 31 St & 36 Ave
Inserting station 3627 at Coming Soon: 31 St & 30 Ave
Inserting station 3628 at Coming Soon: Lenox Ave & W 117 St
Inserting station 3629 at Coming Soon: W 126 St & Adam Clayton Powell Blvd
Inserting station 3630 at Coming Soon: Frederick Douglass Blvd & W 115 St
Inserting station 3631 at Coming Soon: Crown St & Bedford Ave
Inserting station 3632 at E 12 St & Avenue B


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3627' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3628' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3629' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3630' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3631' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:32: Warning: (1062, "Duplicate entry '3632' for key 'PRIMARY'")


In [7]:
# Now we fetch the data about the time varying elements of the citibike stations
from datetime import datetime
results = requests.get(url_status).json() 
data = results["data"]["stations"]

table_name = 'Status'
query_template = '''INSERT IGNORE INTO {db}.{table}(station_id, 
                                            num_bikes_available,
                                            num_bikes_disabled,
                                            num_docks_available,
                                            num_docks_disabled,
                                            is_installed,
                                            is_renting,
                                            is_returning,
                                            last_reported,
                                            eightd_has_available_keys) 
                    VALUES (%s, %s, %s, %s, %s, %s, %s, %s, %s, %s)'''.format(db=db_name, table=table_name)
cursor = con.cursor()

for entry in data:
    station_id = int(entry['station_id'])
    num_bikes_available = entry['num_bikes_available']
    num_bikes_disabled = entry['num_bikes_disabled']
    num_docks_available = entry['num_docks_available']
    num_docks_disabled = entry['num_docks_disabled']
    is_installed = entry['is_installed']
    is_renting = entry['is_renting']
    is_returning = entry['is_returning']
    last_reported = datetime.fromtimestamp(entry['last_reported']) 
    eightd_has_available_keys = entry['eightd_has_available_keys']
                       
    print("Inserting station", station_id)
    query_parameters = (station_id, num_bikes_available, num_bikes_disabled,
                        num_docks_available, num_docks_disabled, is_installed, is_renting,
                        is_returning, last_reported, eightd_has_available_keys)
    cursor.execute(query_template, query_parameters)

con.commit()
cursor.close()

Inserting station 72
Inserting station 79
Inserting station 82
Inserting station 83
Inserting station 116
Inserting station 119
Inserting station 120
Inserting station 127
Inserting station 128
Inserting station 143
Inserting station 144
Inserting station 146
Inserting station 150
Inserting station 151
Inserting station 152
Inserting station 153
Inserting station 157
Inserting station 161
Inserting station 164
Inserting station 167
Inserting station 168
Inserting station 173
Inserting station 174
Inserting station 195
Inserting station 212
Inserting station 216
Inserting station 217
Inserting station 223
Inserting station 224
Inserting station 228
Inserting station 229
Inserting station 232
Inserting station 236
Inserting station 237
Inserting station 238
Inserting station 239
Inserting station 241
Inserting station 242
Inserting station 243
Inserting station 244
Inserting station 245
Inserting station 247
Inserting station 248
Inserting station 249
Inserting station 251
Inserting stat

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '72-2017-09-18 23:21:46' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '82-2017-09-18 23:06:44' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '116-2017-09-18 23:24:47' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '146-2017-09-18 23:24:59' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '150-2017-09-18 23:20:05' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '153-2017-08-28 13:16:10' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '157-2017-09-18 23:24:31' for key 'PRIMA

Inserting station 459
Inserting station 460
Inserting station 461
Inserting station 462
Inserting station 465
Inserting station 466
Inserting station 467
Inserting station 468
Inserting station 469
Inserting station 470
Inserting station 471
Inserting station 472
Inserting station 473
Inserting station 474
Inserting station 476
Inserting station 477
Inserting station 478
Inserting station 479
Inserting station 480
Inserting station 481
Inserting station 482
Inserting station 483
Inserting station 484
Inserting station 485
Inserting station 486
Inserting station 487
Inserting station 488
Inserting station 490
Inserting station 491
Inserting station 492
Inserting station 494
Inserting station 495
Inserting station 496
Inserting station 497
Inserting station 498
Inserting station 499
Inserting station 500
Inserting station 501
Inserting station 502
Inserting station 503
Inserting station 504
Inserting station 505
Inserting station 507
Inserting station 508
Inserting station 509
Inserting 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '462-2017-09-18 23:24:15' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '465-2017-09-18 23:23:19' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '468-2017-09-18 23:21:43' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '469-2017-09-18 23:20:43' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '471-2017-09-18 23:23:59' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '473-2017-09-18 23:18:52' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '481-2017-09-18 23:25:10' for key 'PRI

Inserting station 3151
Inserting station 3152
Inserting station 3155
Inserting station 3156
Inserting station 3157
Inserting station 3158
Inserting station 3159
Inserting station 3160
Inserting station 3161
Inserting station 3162
Inserting station 3163
Inserting station 3164
Inserting station 3165
Inserting station 3166
Inserting station 3167
Inserting station 3168
Inserting station 3169
Inserting station 3170
Inserting station 3171
Inserting station 3172
Inserting station 3173
Inserting station 3175
Inserting station 3176
Inserting station 3177
Inserting station 3178
Inserting station 3179
Inserting station 3180
Inserting station 3182
Inserting station 3183
Inserting station 3184
Inserting station 3185
Inserting station 3186
Inserting station 3187
Inserting station 3188
Inserting station 3189
Inserting station 3190
Inserting station 3191
Inserting station 3192
Inserting station 3193
Inserting station 3194
Inserting station 3195
Inserting station 3196
Inserting station 3197
Inserting s

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3151-2017-09-18 23:25:25' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3155-2017-09-01 13:02:07' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3157-2017-09-18 23:22:27' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3158-2017-09-18 23:08:05' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3160-2017-09-18 23:17:36' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3161-2017-09-18 23:14:02' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3165-2017-09-18 23:24:44' for k

 3371
Inserting station 3372
Inserting station 3373
Inserting station 3374
Inserting station 3375
Inserting station 3376
Inserting station 3377
Inserting station 3378
Inserting station 3379
Inserting station 3381
Inserting station 3382
Inserting station 3383
Inserting station 3384
Inserting station 3386
Inserting station 3387
Inserting station 3388
Inserting station 3389
Inserting station 3390
Inserting station 3391
Inserting station 3392
Inserting station 3393
Inserting station 3394
Inserting station 3395
Inserting station 3396
Inserting station 3397
Inserting station 3398
Inserting station 3399
Inserting station 3400
Inserting station 3401
Inserting station 3402
Inserting station 3403
Inserting station 3404
Inserting station 3405
Inserting station 3407
Inserting station 3408
Inserting station 3409
Inserting station 3410
Inserting station 3411
Inserting station 3412
Inserting station 3413
Inserting station 3414
Inserting station 3415
Inserting station 3416
Inserting station 3417
Inser

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3387-2017-09-18 22:38:06' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3389-2017-09-18 23:20:52' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3390-2017-09-18 23:10:49' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3391-2017-09-18 22:57:51' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3392-2017-09-18 22:59:36' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3393-2017-09-18 23:03:50' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3394-2017-09-18 23:13:57' for k


Inserting station 3576
Inserting station 3577
Inserting station 3578
Inserting station 3579
Inserting station 3580
Inserting station 3581
Inserting station 3582
Inserting station 3583
Inserting station 3584
Inserting station 3585
Inserting station 3586
Inserting station 3587
Inserting station 3588
Inserting station 3589
Inserting station 3590
Inserting station 3591
Inserting station 3592
Inserting station 3593
Inserting station 3594
Inserting station 3595
Inserting station 3596
Inserting station 3597
Inserting station 3598
Inserting station 3599
Inserting station 3600
Inserting station 3601
Inserting station 3602
Inserting station 3603
Inserting station 3604
Inserting station 3605
Inserting station 3606
Inserting station 3607
Inserting station 3608
Inserting station 3609
Inserting station 3610
Inserting station 3611
Inserting station 3612
Inserting station 3613
Inserting station 3614
Inserting station 3615
Inserting station 3616
Inserting station 3617
Inserting station 3618
Inserting 

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3579-1970-01-01 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3580-1970-01-01 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3581-1970-01-01 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3582-1970-01-01 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3583-1970-01-01 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3584-1970-01-01 00:00:00' for key 'PRIMARY'")
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:36: Warning: (1062, "Duplicate entry '3585-1970-01-01 00:00:00' for k